In [41]:
import pickle
import os
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"
MLFLOW_EXPERIMENT_NAME = "homework_week_2"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

2024/05/28 18:41:55 INFO mlflow.tracking.fluent: Experiment with name 'homework_week_2' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/yorgos/mlops-zoomcamp/Week_2/artifacts_local/2', creation_time=1716914515274, experiment_id='2', last_update_time=1716914515274, lifecycle_stage='active', name='homework_week_2', tags={}>

## Question 1

In [5]:
!mlflow --version

mlflow, version 2.13.0


## Question 2

In [7]:
output_files = os.listdir("output")
print(f"There are {len(output_files)} files in the output directory: {output_files}")

There are 4 files in the output directory: ['dv.pkl', 'val.pkl', 'test.pkl', 'train.pkl']


## Question 3

In [37]:
MLFLOW_EXPERIMENT_NAME = "homework_week_2"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# Get the experiment_id using the experiment name
experiment = client.get_experiment_by_name("homework_week_2")
experiment_id = experiment.experiment_id

# Get the run_id of the first (and only) experiment run
runs = client.search_runs(experiment_ids=experiment_id)
run_id = runs[0].info.run_id

# Load the model from the local artifact store
artifact_path = f"mlartifacts/{experiment_id}/{run_id}/artifacts/model/model.pkl"
with open(artifact_path, "rb") as f:
    model = pickle.load(f)

print(f"min_samples_split: {model.min_samples_split}")

min_samples_split: 2


## Question 4

CLI command: `mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts`

## Question 5

In [80]:
# Get the experiment_id using the experiment name
experiment = client.get_experiment_by_name("random-forest-hyperopt")

# Get the run_id of the experiment with the best validation RMSE
best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        max_results=1,
        order_by=["metrics.rmse ASC"]
)[0]

metric = client.get_metric_history(runs[0].info.run_id, "rmse")
print(f"The best validation RMSE is {best_run.data.metrics['rmse']}")

The best validation RMSE is 5.335419588556921


## Question 6

In [81]:
experiment = client.get_experiment_by_name("random-forest-best-models")
best_run = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.test_rmse ASC"]    
)[0]

best_run_id = best_run.info.run_id
best_run_test_rmse = best_run.data.metrics["test_rmse"]
print(f"The RMSE of the best model on the test set is {best_run_test_rmse}")

The RMSE of the best model on the test set is 5.567408012462019
